In [1]:
import seaborn as sns
import numpy as np
import datetime as dt
import pandas as pd
import matplotlib.pyplot as plt
import math
from sklearn import preprocessing
import nltk as nltk
import cv2
import imageio
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
from nltk.draw.dispersion import dispersion_plot
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import KFold 
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import log_loss
from sklearn.metrics import f1_score
from sklearn.ensemble import AdaBoostClassifier
from sklearn.datasets import make_classification
from sklearn.datasets import make_blobs
from sklearn.neighbors import NearestCentroid
from sklearn.naive_bayes import BernoulliNB
from sklearn.impute import SimpleImputer


from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
import warnings
warnings.filterwarnings("ignore")
#                         category=DeprecationWarning)


pd.set_option('display.max_rows', None)
df = pd.read_json('C:/Users/User/Desktop/CMPT459/as1/train.json')
df_testSet = pd.read_json('C:/Users/User/Desktop/CMPT459/as1/test.json', dtype={'listing_id': 'str'})

In [13]:
df_testSet = pd.read_json('C:/Users/User/Desktop/CMPT459/as1/test.json', dtype={'listing_id': 'str'})
listingID = df_testSet['listing_id']

## TRAINING


### Preprocessing

In [14]:
df_price = df['price']  
IQR = df_price.quantile(0.75)-df_price.quantile(0.25)
upper = df_price.quantile(0.75)+IQR*1.5
lower = df_price.quantile(0.25)-IQR*1.5 

df1 = df[(df['price'] <upper) & (df['price'] >lower)]

df_lat = df1['latitude']

IQR = df_lat.quantile(0.75)-df_lat.quantile(0.25)
upper = df_lat.quantile(0.75)+IQR*1.5
lower = df_lat.quantile(0.25)-IQR*1.5
 
df2 = df1[(df1['latitude']<upper) & (df1['latitude']>lower)]

df_long = df2['longitude']
 
IQR = df_long.quantile(0.75)-df_long.quantile(0.25)
upper = df_long.quantile(0.75)+IQR*1.5
lower = df_long.quantile(0.25)-IQR*1.5 
df3 = df2[(df2['longitude'] >lower) & (df2['longitude']<upper)]

df_bedrooms = df3['bedrooms']
 
IQR = df_bedrooms.quantile(0.75)-df_bedrooms.quantile(0.25)
upper = df_bedrooms.quantile(0.75)+IQR*1.5
lower = max(df_bedrooms.quantile(0.25)-IQR*1.5, 0)
 
df4 = df3[(df3['bedrooms'] >= lower) & (df3['bedrooms'] <= upper)]

df_bathrooms = df4['bathrooms']
 
IQR = df_bathrooms.quantile(0.75)-df_bathrooms.quantile(0.25)

upper = math.ceil(df_bathrooms.quantile(0.75)+IQR*1.5)
lower = max(df_bathrooms.quantile(0.25)-IQR*1.5, 0)
average = sum(df_bathrooms) / len(df_bathrooms)

df5 = df4[(df4['bathrooms'] >= lower) & (df4['bathrooms'] <= upper)]

df6 = df5.drop(['listing_id','features','description', 'building_id', 'manager_id', 'building_id',
                'created', 'display_address', 'street_address', 'photos', 'interest_level'], axis=1)
X=df6
y = df5["interest_level"] 

X.head()


,bathrooms,bedrooms,latitude,longitude,price
4,1.0,1,40.7108,-73.9539,2400
6,1.0,2,40.7513,-73.9722,3800
9,1.0,2,40.7575,-73.9625,3495
15,1.0,0,40.7439,-73.9743,2795
19,1.0,0,40.7769,-73.9467,1945


### Analysis

In [15]:
## predictions are in the order of HIGH LOW MEDIUM 
# Split labeled data into 60% training dataset and 40% test dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)
  

clf = BernoulliNB(alpha=0, fit_prior=False)
clf.fit(X, y)
scores = cross_val_score(clf, X, y, cv=5)

# Predict labels on the test data using the trained classifier.
predictions = clf.predict(X_test)

# Calculate the F1 score by comparing predicted and actual labels.
F1 = f1_score(y_test, predictions, average='weighted')

y_prob = clf.predict_proba(X_test) 
logloss = log_loss(y_test, y_prob)


print(np.mean(scores))
print(F1)
print(logloss)


0.2015875929060586
0.09443465723237396
1.0983198982550257


## Testing

In [16]:
listingID = df_testSet['listing_id']
df_testSet.shape

listingID.head()

0    7142618
1    7210040
2    7174566
3    7191391
5    7171695
Name: listing_id, dtype: object

In [17]:
df_testSet = df_testSet.drop(columns=['listing_id','building_id','features','description','photos', 'manager_id','created','display_address','street_address'])
,'description', 'building_id', 'manager_id','created', 'display_address', 'street_address', 'photos'], axis=1) 

('description',
 ("'building_id',",
  "'manager_id','created',",
  "'display_address',",
  "'street_address',",
  "'photos'],",
  'axis=1)'))

In [18]:
df_testSet.head()

,bathrooms,bedrooms,latitude,longitude,price
0,1.0,1,40.7185,-73.9865,2950
1,1.0,2,40.7278,-74.0000,2850
2,1.0,0,40.7260,-74.0026,2295
3,1.0,2,40.7321,-74.0028,2900
5,1.0,1,40.7054,-74.0095,3254


In [19]:
Xt = df_testSet  
 
high = [] 
low=[]
medium = []
 
    
for i in range(0,Xt.shape[0]): 
    h = clf.predict_proba(Xt[i:i+1])[0][0]
    l = clf.predict_proba(Xt[i:i+1])[0][1]
    m = clf.predict_proba(Xt[i:i+1])[0][2]

    high.append(h)
    low.append(l)
    medium.append(m)  
    

In [20]:
df_test = pd.DataFrame()  
df_test['listing_id'] = listingID 
df_test['high'] = high
df_test['medium'] = medium
df_test['low'] = low

df_test.head()


,listing_id,high,medium,low
0,7142618,0.327818,0.339415,0.332767
1,7210040,0.327818,0.339415,0.332767
2,7174566,0.350283,0.314642,0.335075
3,7191391,0.327818,0.339415,0.332767
5,7171695,0.327818,0.339415,0.332767


In [21]:
df_test.to_csv(r'C:\Users\User\Desktop\CMPT459\M3\submission_bayes_first2.csv', index = False)

## Modification

In [24]:
## predictions are in the order of HIGH LOW MEDIUM 
# Split labeled data into 60% training dataset and 40% test dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)
  

clf = BernoulliNB(alpha=1.0, fit_prior=True)
clf.fit(X, y)
scores = cross_val_score(clf, X, y, cv=5)

# Predict labels on the test data using the trained classifier.
predictions = clf.predict(X_test)

# Calculate the F1 score by comparing predicted and actual labels.
F1 = f1_score(y_test, predictions, average='weighted')

y_prob = clf.predict_proba(X_test) 
logloss = log_loss(y_test, y_prob)


print(np.mean(scores))
print(F1)
print(logloss) 

0.6856287481395004
0.5604376770946726
0.7995295891571709


In [34]:
Xt = df_testSet  
 
high = [] 
low=[]
medium = []
 
    
for i in range(0,Xt.shape[0]): 
    h = clf.predict_proba(Xt[i:i+1])[0][0]
    l = clf.predict_proba(Xt[i:i+1])[0][1]
    m = clf.predict_proba(Xt[i:i+1])[0][2]
    
    high.append(h)
    low.append(l)
    medium.append(m)  
    

In [35]:
df_test = pd.DataFrame()  
df_test['listing_id'] = listingID 
df_test['high'] = high
df_test['medium'] = medium
df_test['low'] = low

df_test.head()


,listing_id,high,medium,low
0,7142618,0.078172,0.236083,0.685745
1,7210040,0.078172,0.236083,0.685745
2,7174566,0.078172,0.236083,0.685745
3,7191391,0.078172,0.236083,0.685745
5,7171695,0.078172,0.236083,0.685745


In [26]:
df_test.to_csv(r'C:\Users\User\Desktop\CMPT459\M3\submission_bayes7.csv', index = False)